In [1]:
"""
일급 함수 디자인 패턴

- 일급 함수를 지원하는 언어에서는 전략, 명령, 템플릿 메서드, 비지터 패턴을 다시 생각해봐야함
- 함수 객체를 이용해서 전략 패턴을 리팩토링, 비슷한 방법으로 명령 패턴을 단순화하는 과정 학습
"""


'\n일급 함수 디자인 패턴\n\n- 일급 함수를 지원하는 언어에서는 전략, 명령, 템플릿 메서드, 비지터 패턴을 다시 생각해봐야함\n- 함수 객체를 이용해서 전략 패턴을 리팩토링, 비슷한 방법으로 명령 패턴을 단순화하는 과정 학습\n'

In [11]:
"""
파이썬으로 전략패턴 구현
- context : Order
- Strategy : Promotion(ABC)
- ConcreteStrategy : FidelityPromo, BulkItemPromo, LargeOrderPromo
"""

from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem : 
    def __init__(self, product, quantity, price) : 
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self) :
        return self.price * self.quantity
    
class Order : # 콘텍스트
    
    def __init__(self, customer, cart, promotion=None) : 
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self) : 
        if not hasattr(self, '__total') :
            self.__total = sum(item.total() for item in self.cart)
            
        return self.__total
    
    
    def due(self) :
        if self.promotion is None : 
            discount = 0
            
        else : 
            discount = self.promotion.discount(self)
            
        return self.total() - discount
    
    def __repr__(self) : 
        fmt = '<Order total : {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    

class Promotion(ABC) : # 전략 : 추상 베이스 클래스 
        
    @abstractmethod
    def discount(self, order) : 
        """할인액을 구체적인 숫자로 변환함"""
            
            
class FidelityPromo(Promotion) : # 첫 번째 구체적인 전략 
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
        
    def discount(self, order) : 
        return order.total() * 0.5 if order.customer.fidelity >= 1000 else 0
        
class BulkItemPromo(Promotion) : # 두 번째 구체적인 전략 
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
        
    def discount(self, order) : 
        discount = 0
        for item in order.cart : 
            if item.quantity >= 20 : 
                discount += item.total() * .1
                    
        return discount
        
class LargeOrderPromo(Promotion) : # 세 번째 구체적인 전략 
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
        
    def discount(self, order) : 
        distinct_items = {item.product for item in order.cart} 
            
        if len(distinct_items) >= 10 :
            return order.total() * .07
            
        return 0

In [13]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, .5), 
    LineItem('apple', 10, 1.5), 
    LineItem('watermellon', 5, 5.0)]
        
Order(joe, cart, FidelityPromo())
Order(ann, cart, FidelityPromo())


<Order total : 42.00 due: 21.00>

In [14]:
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total : 30.00 due: 28.50>

In [19]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total : 10.00 due: 9.30>

In [21]:
Order(joe, cart, LargeOrderPromo())

<Order total : 42.00 due: 42.00>

In [58]:
"""
위의 코드 리팩토링

- 구체적인 전략 객체가 일반 함수처럼 보임 -> 함수를 객체로 사용해서 더 적은 코드로 동일한 기능 처리

전랸 패턴의 단점 
- '런타임 비용'. 런타임시에 매번 전략 클래스를 생성해서 사용해야한다.
- 즉, 전략 패턴의 런타임 비용을 극복하기 위해 플라이웨이트 패턴을 사용하도록 권고함
- 일반 함수도 여러 콘텍스트에서 동시에 공유할 수 있는 공유 객체임

"""

class LineItem : 
    def __init__(self, product, quantity, price) : 
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self) :
        return self.price * self.quantity
    
class Order : # 콘텍스트
    
    def __init__(self, customer, cart, promotion=None) : 
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self) : 
        if not hasattr(self, '__total') :
            self.__total = sum(item.total() for item in self.cart)
            
        return self.__total
    
    
    def due(self) :
        if self.promotion is None : 
            discount = 0
            
        else : 
            discount = self.promotion(self)
            
        return self.total() - discount
    
    def __repr__(self) : 
        fmt = '<Order total : {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
    
def fidelity_promo(order) : 
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order) :
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    
    for i in order.cart : 
        if i.quantity >= 20 : 
            discount += i.total() * .1
            
    return discount

def large_order_promo(order) :
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = { i.product for i in order.cart }
    
    if len(distinct_items) >= 10 : 
        return order.total() * .07
    
    return 0



In [59]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, .5), 
    LineItem('apple', 10, 1.5), 
    LineItem('watermellon', 5, 5.0)]

In [60]:
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]

In [61]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [62]:
"""
'메타 전략' 생성

- 모든 할인을 적용해서 가장 큰 값을 반환하는 best_promo 함수
- 해당 함수의 한계, 새로운 할인 함수를 고려하지 않음 
- 파이썬 모듈(파일)도 일급 객체

"""

""" 문제점 : 추후에 새로운 할인 함수를 고려 못함 """
promos = [fidelity_promo, bulk_item_promo, large_order_promo] # Flyweight 패턴 

""" 1. 해결책 - 글로벌 네임 스페이스에서 xxx_promo 등록된 함수 객체를 찾음 """
promos = [globals()[name] for name in globals() 
              if name.endswith('_promo') 
              and name != 'best_promo']

""" 2. 해결책 - promotions라는 별도 모듈에 내부 조사를 수행해서 만든 전략 함수 리스트를 사용하도록 변경 
- 주어진 주문에 대해 할인액을 계산하는 함수들만 promotions 모듈에 넣으면 된다.
"""
# import inspect, promotions

# promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]


def best_promo(order) : 
    """최대로 할인받을 금액을 반환함"""
    
    return max(promo(order) for promo in promos)




In [63]:
Order(joe, banana_cart, best_promo)

<Order total : 30.00 due: 28.50>

In [64]:
Order(joe, banana_cart, best_promo)

<Order total : 30.00 due: 28.50>

In [65]:
Order(ann, cart, best_promo)

<Order total : 42.00 due: 39.90>

In [71]:
"""
함수를 인수로 전달하는 기법을 사용하면 명령 디자인 패턴도 구현을 단순화 할 수 있음
- Command 패턴의 목적 : 연산을 실행하는 객체(호출차, Invoker)와 연산을 구현하는 객체(수신자, Receiver)를 분리함
- 명령 객체를 수신자와 호출자 사이에 놓고, 명령은 execute() 라는 단 하나의 메서드로 인터페이스를 구현함 
- Command 객체 대신 간단히 함수를 바로 지정할 수 있음, 호출자는 command()로 사용할 수 있음. MacroCommand의 객체는 콜러블이 됨
"""

class Menu : 
    
    def __init__(self, macro_command) : 
        self.macro_command = macro_command
        
    def show_total_order(self) :
        print("== show total command!! ==")
        self.macro_command()
        

class MacroCommand : 
    """명령 리스트를 실행하는 명려"""
    
    def __init__(self, commands) :
        self.commands = list(commands)
        
    def __call__(self) : 
        for c in self.commands : 
            c()
            
# 하나의 Command
def order_pizza_command() : 
    print("order pizza")
    
def order_chiken_command() : 
    print("order chiken")
    
def order_coke_command() :
    print("order coke")
    
commands = [globals()[name] for name in globals() if name.endswith('_command') ]
macro = MacroCommand(commands)
menu = Menu(macro)
menu.show_total_order()

== show total command!! ==
hi
hello
order pizza
order chiken
order coke


In [72]:
"""
- 단일 메서드 인터페이스를 구현한 클래스의 객체를 콜러블로 대체하는 것임
- 모든 파이썬 콜러블이 __call__() 이라는 단일 메서드 인터페이스를 구현함
"""

'\n- 단일 메서드 인터페이스를 구현한 클래스의 객체를 콜러블로 대체하는 것임\n- 모든 파이썬 콜러블이 __call__() 이라는 단일 메서드 인터페이스를 구현함\n'